In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataloaders.midair import DataLoaderMidAir as MidAir
from dataloaders import DataloaderParameters

param = DataloaderParameters({'_comment': '/Users/pascalleroy/Documents/m4depth/M4Depth/relative paths should be written relative to this file',
                                 'midair': '/Users/pascalleroy/Documents/m4depth/M4Depth/datasets/MidAir',
                                 'kitti-raw': '/Users/pascalleroy/Documents/m4depth/M4Depth/datasets/Kitti',
                                 'tartanair': '/Users/pascalleroy/Documents/m4depth/M4Depth/datasets/TartanAir'},
                             'data/midair/small_train_data',
                             8, #db_seq_len
                             4, # seq_len
                             True)
print(param)
chosen_dataloader = MidAir()
chosen_dataloader.get_dataset("train", param, batch_size=4)
data = chosen_dataloader.dataset

DataloaderParameters(db_path_config={'_comment': '/Users/pascalleroy/Documents/m4depth/M4Depth/relative paths should be written relative to this file', 'midair': '/Users/pascalleroy/Documents/m4depth/M4Depth/datasets/MidAir', 'kitti-raw': '/Users/pascalleroy/Documents/m4depth/M4Depth/datasets/Kitti', 'tartanair': '/Users/pascalleroy/Documents/m4depth/M4Depth/datasets/TartanAir'}, records_path='data/midair/small_train_data', db_seq_len=8, seq_len=4, augment=True)


2022-09-12 17:21:27.688913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from importlib import reload
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras as ks
from keras.utils import vis_utils
from metrics import AbsRelError, SqRelError, RootMeanSquaredError, \
    RootMeanSquaredLogError, ThresholdRelError

from callbacks import CustomCheckpointCallback

print(tf.__version__)
from m4depth_network import M4Depth, M4depthAblationParameters

tf.random.set_seed(42)

ablation_settings = M4depthAblationParameters()

model = M4Depth(depth_type=chosen_dataloader.depth_type,
                nbre_levels=6,
                ablation_settings=ablation_settings,
                is_training=True)

# Initialize callbacks
tensorboard_cbk = ks.callbacks.TensorBoard(
    log_dir="log_dir", histogram_freq=1200, write_graph=True,
    write_images=False, update_freq=1200,
    profile_batch=0, embeddings_freq=0, embeddings_metadata=None)
weights_dir = os.path.join("pretrained_weights/midair/", "train")
model_checkpoint_cbk = CustomCheckpointCallback(weights_dir, resume_training=True)

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=opt, metrics=[RootMeanSquaredLogError()])
nbre_epochs = 2

model.fit(data, epochs= nbre_epochs + 1,
          initial_epoch=model_checkpoint_cbk.resume_epoch,
          callbacks=[tensorboard_cbk, model_checkpoint_cbk])

2.9.1
Could not import cuda Backproject Module, using python implementation
Warrning, importing functionnal version of utils.
Checkpoint save directory: pretrained_weights/midair/train
Proceeding with scratch network initialization
Epoch 1/3
Seq sample 0
Skipping temporal memory instanciation
Skipping temporal memory instanciation
Skipping temporal memory instanciation
Skipping temporal memory instanciation
Skipping temporal memory instanciation
Skipping temporal memory instanciation
Seq sample 1
get_disparity_sweeping_cv Tensor("DSCV/StatefulPartitionedCall:0", shape=(4, 6, 6, 3, 1), dtype=float32)
get_disparity_sweeping_cv Tensor("DSCV/StatefulPartitionedCall:0", shape=(4, 12, 12, 3, 1), dtype=float32)
get_disparity_sweeping_cv Tensor("DSCV/StatefulPartitionedCall:0", shape=(4, 24, 24, 3, 1), dtype=float32)
get_disparity_sweeping_cv Tensor("DSCV/StatefulPartitionedCall:0", shape=(4, 48, 48, 3, 1), dtype=float32)
get_disparity_sweeping_cv Tensor("DSCV/StatefulPartitionedCall:0", shape

KeyboardInterrupt: 